### General map making function to build file from data

In [105]:
import numpy as np

def make_coastal(name, locations):
    f = open(name, "w")
    population = 333333
    sub_pop = 222222
    num_parties = 3
    prefs_left = [1.0,0.1,0.1]
    prefs_right = [0.1,1.0,0.1]
    line = str(population) + " " + str(num_parties) + "\n"
    f.write(line)
    for i in range(sub_pop):
        location = locations[i]
        loc_x = location[0]
        loc_y = location[1]
        prefs = prefs_right
        if loc_x < 500:
            prefs = prefs_left
        f.write(str(loc_x) + " ")
        f.write(str(loc_y) + " ")
        f.write(str(prefs[0]) + " " + str(prefs[1]) + " " + str(prefs[2]) + "\n")

    f.close()

In [106]:
def make_central(name):
    mean = np.array([centroid.x + 50, centroid.y])
    var = 200
    cov = np.array([[var, 0],
                    [0, var]])

    f = open(name, "a+")
    population = 111111
    num_parties = 3
    prefs = [0.1,0.1,1.0]
    for i in range(population):
        location = np.random.multivariate_normal(mean, cov, size=1)
        loc_x = location[0][0]
        loc_y = location[0][1]
        f.write(str(loc_x) + " ")
        f.write(str(loc_y) + " ")
        f.write(str(prefs[0]) + " " + str(prefs[1]) + " " + str(prefs[2]) + "\n")
    f.close()

### Geometry functions

In [107]:
# get a point on a side from distance along that side
def left_y(x):
    if x > 500 or x < 0:
        return None
    return x * np.sqrt(3)

def get_left_point(d):
    return (d / 2, d / 2 * np.sqrt(3))
    
def get_right_point(d):
    l_point = get_left_point(d)
    return (1000 - l_point[0], l_point[1])

def get_bottom_point(d):
    return (d, 0)

get_point = {0: get_bottom_point, 1: get_left_point, 2: get_right_point}

# functions to get points on sides given x
def y_bottom(x):
    return 0
def y_left(x):
    return x * np.sqrt(3)
def y_right(x):
    return np.sqrt(3) * (1000 - x)

# check bounds
def in_bounds(point):
    x, y = point[0], point[1]
    condition1 = y >= y_bottom(x)
    condition2 = y <= y_left(x)
    condition3 = y <= y_right(x)
    return condition1 and condition2 and condition3

# functions to move away from a side of the triangle
def right_side_deltas(d):
    return (-d/2 * np.sqrt(3), -d/2)
def left_side_deltas(d):
    return (d/2 * np.sqrt(3), -d/2)
def bottom_side_deltas(d):
    return (0, d)

get_deltas = {0: bottom_side_deltas, 1: left_side_deltas, 2: right_side_deltas}

### Get voter points and voter preferences

In [108]:
def generate_fuzzy_coastal_point():
    while(True):
        side = np.random.randint(3)
        d = np.random.normal(500, 300)
        border_point = get_point[side](d)
        dis_in = np.random.exponential() * 50
        deltas = get_deltas[side](dis_in)
        point = (border_point[0] + deltas[0], border_point[1] + deltas[1])
        if in_bounds(point):
            return point

locs = []
for i in range(222222):
    locs.append(generate_fuzzy_coastal_point())

In [109]:
make_coastal("utah1", locs)

In [110]:
make_central("utah1")

### Print map to file

In [18]:
make_map("map2", locs, preferences)

In [37]:
class Point:
    def __init__(self, x,y):
        self.x = x
        self.y = y
    def __repr__(self):
        return "({}, {})".format(self.x, self.y)

In [38]:
map_points = [Point(0,0), Point(1000,0), Point(500, 500 * np.sqrt(3))]
mean_x = np.mean([map_points[0].x, map_points[1].x, map_points[2].x])
mean_y = np.mean([map_points[0].y, map_points[1].y, map_points[2].y])
centroid = Point(mean_x, mean_y)

In [46]:
x_linspace = np.linspace(0,1000, 28)
triangles_bottom = []
for i in range(27):
    triangle = [centroid]
    triangle.append(Point(x_linspace[i], 0))
    triangle.append(Point(x_linspace[i+1], 0))
    triangles_bottom.append(triangle)
    
    

In [48]:
x_linspace = np.linspace(0, 500, 28)
triangles_left = []
for i in range(27):
    triangle = [centroid]
    x = x_linspace[i]
    y = y_left(x)
    triangle.append(Point(x, y))
    x = x_linspace[i + 1]
    y = y_left(x)
    triangle.append(Point(x, y))
    triangles_left.append(triangle)
    
    

In [51]:
x_linspace = np.linspace(500, 1000, 28)
triangles_right = []
for i in range(27):
    triangle = [centroid]
    x = x_linspace[i]
    y = y_right(x)
    triangle.append(Point(x, y))
    x = x_linspace[i + 1]
    y = y_right(x)
    triangle.append(Point(x, y))
    triangles_right.append(triangle)

In [49]:
def make_triangles_file(name, triangles):
    f = open(name, "w")
    num_poly = len(triangles)
    line = str(num_poly) + "\n"
    f.write(line)
    for triangle in triangles:
        f.write(str(3) + " ")
        for point in triangle:
            f.write(str(point.x) + " " + str(point.y) + " ")
        f.write("\n")
    f.close()

In [52]:
make_triangles_file("triangles_right", triangles_right)